In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax

os.chdir('C:/Users/christian fink/OneDrive/Math485')
df = pd.read_csv('DisasterDeclarationsSummaries.csv')



C:\Users\christian fink\AppData\Local\Temp\ipykernel_1804\1586937123.py:16: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('DisasterDeclarationsSummaries.csv')


In [21]:
df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5526-CO,5526,CO,FM,2024-08-01T00:00:00.000Z,2024,Fire,QUARRY FIRE,0,0,...,99059,Jefferson (County),24106,NaN,2024080102,8,R,2024-08-27T18:22:14.800Z,e753ba692156f389dbe19f7a1c332d04ae145f74,17c24d4a-49a9-4cac-9322-e5427c4cdfeb


In [23]:
state_column = 'state'
state_count = df[state_column].value_counts()



In [24]:
state_count

state
TX    5350
KY    2762
MO    2750
FL    2644
LA    2586
GA    2559
OK    2543
VA    2531
NC    2297
PR    2078
MS    1945
IA    1922
KS    1839
AL    1742
TN    1709
CA    1678
AR    1624
MN    1621
NE    1533
NY    1532
SD    1466
IN    1464
ND    1393
IL    1313
OH    1292
WV    1266
PA    1243
SC    1098
ME    1065
WA    1014
WI     892
MI     805
CO     664
MT     640
NJ     626
OR     622
NM     530
MD     447
MA     414
VT     405
ID     369
AZ     342
AK     333
NH     320
NV     285
CT     264
UT     255
WY     132
RI     123
HI     108
VI      84
MP      76
AS      76
DE      53
MH      53
FM      31
DC      23
GU      22
PW       1
Name: count, dtype: int64

In [34]:
columns_to_drop = ['femaDeclarationString',
                'declarationType', 'declarationDate', 'ihProgramDeclared', 'iaProgramDeclared',
                'paProgramDeclared', 'hmProgramDeclared', 'incidentBeginDate', 'incidentEndDate',
                'disasterCloseoutDate', 'tribalRequest', 'fipsStateCode', 'fipsCountyCode', 'declarationRequestNumber',
                'placeCode', 'lastIAFilingDate', 'incidentId', 'region', 'designatedIncidentTypes', 'lastRefresh',
                'hash', 'id'] 
df.drop(columns=columns_to_drop)

,disasterNumber,state,fyDeclared,incidentType,declarationTitle,designatedArea
0,5530,NV,2024,Fire,GOLD RANCH FIRE,Washoe (County)
1,5529,OR,2024,Fire,LEE FALLS FIRE,Washington (County)
2,5528,OR,2024,Fire,ELK LANE FIRE,Jefferson (County)
3,5527,OR,2024,Fire,MILE MARKER 132 FIRE,Deschutes (County)
4,5526,CO,2024,Fire,QUARRY FIRE,Jefferson (County)
...,...,...,...,...,...,...
66849,9,TX,1953,Flood,FLOOD,Statewide
66850,8,IA,1953,Flood,FLOOD,Statewide
66851,7,MA,1953,Tornado,TORNADO,Statewide
66852,2,TX,1953,Tornado,TORNADO & HEAVY RAINFALL,Statewide


In [57]:
df.groupby(['state', 'incidentType']).size().reset_index(name='count')

,state,incidentType,count
0,AK,Biological,121
1,AK,Coastal Storm,2
2,AK,Earthquake,13
3,AK,Fire,31
4,AK,Flood,60
...,...,...,...
513,WY,Flood,25
514,WY,Severe Ice Storm,2
515,WY,Severe Storm,21
516,WY,Tornado,2


In [76]:
grouped_df = df.groupby(['state', 'incidentType']).size().reset_index(name='count')
danger_ratings = {
    'Fire': 6,
    'Flood': 8,
    'Tornado': 7,
    'Hurricane': 10,
    'Earthquake': 9}
grouped_df['dangerRating'] = grouped_df['incidentType'].map(danger_ratings)
grouped_df['totalRating'] = grouped_df['count'] * grouped_df['dangerRating']
pivot_df = grouped_df.pivot_table(index='state', columns='incidentType', values='totalRating', aggfunc='sum', fill_value = 0)

columns = ['Biological', 'Chemical', 'Coastal Storm', 'Dam/Levee Break', 'Drought', 'Fishing Losses',
           'Snowstorm', 'Straight-Line Winds', 'Freezing', 'Mud/Landslide', 'Other', 
           'Severe Ice Storm', 'Severe Storm', 'Terrorist', 'Tsunami', 'Typhoon', 'Volcanic Eruption',
           'Winter Storm', 'Human Cause', 'Toxic Substances', 'Tropical Storm']
pivot_df = pivot_df.drop(columns = columns)
numeric_columns = pivot_df.columns.difference(['State'])
pivot_df[numeric_columns] = pivot_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df.rename(columns={'state': 'State', 'incidentType':'IncidentType'}, inplace=True)

valid_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 
    'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 
    'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 
    'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]
pivot_df = pivot_df[pivot_df.index.isin(valid_states)]
pivot_df



incidentType,Earthquake,Fire,Flood,Hurricane,Tornado,Total
state,,,,,,
AK,117.0,186.0,480.0,0.0,0.0,783.0
AL,0.0,66.0,832.0,6780.0,434.0,8112.0
AR,0.0,0.0,2168.0,1680.0,791.0,4639.0
AZ,0.0,564.0,656.0,150.0,0.0,1370.0
CA,288.0,2826.0,3592.0,640.0,0.0,7346.0
CO,0.0,930.0,1280.0,0.0,189.0,2399.0
CT,0.0,0.0,136.0,960.0,35.0,1131.0
DE,0.0,0.0,24.0,220.0,0.0,244.0
FL,0.0,1662.0,592.0,12850.0,266.0,15370.0
